In [1]:
import sys
import numpy as np
import pandas as pd
from multiprocessing import Process
sys.path.insert(0, '../../')

In [2]:
from library.notebook_api.data_loader import CombinedDataLoader, ModelDataLoader

In [3]:
from library.source_data.feature_extractor import AudioFeatureExtractor
from library.source_data.parallel_processor import AudioParallelProcessor


## Load initial Meta

In [4]:
#instantate data_loader and the dataframes it makes available 
data_loader = CombinedDataLoader()
df = data_loader.df
df_files_available = data_loader.df_files_available
df_genres_available = data_loader.df_genres_available
df_filtered = data_loader.df_filtered

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [6]:
df_filtered.count()

track_id          9000
dataset           9000
audio_path        9000
label             9000
fma_genre_top     9000
fma_genres        9000
fma_genres_all    9000
file_available    9000
dtype: int64

## Feature Extraction on full dataset with fma_small

Latest approach is to process via a parallel processor which runs multiple extraction tasks in parallel 
and puts outputs to separate parquet folders that are all loadable with data_loader

In [6]:
#instantiate source data df to process
batch_input = df_filtered.sample(100).copy() #subset of full data for demostration
#batch_input = df_filtered.copy()#full data 

#instantate a parallel processing batch 
batch = AudioParallelProcessor(batch_input,version = '006', batch=1, threads = 6)

In [7]:
#sample with 100 and 6 threads 
#execute the parallel processes 
batch.execute()

populate thread process for thread 1  0 16 16
populate thread process for thread 2  17 32 15
populate thread process for thread 3  33 48 15
populate thread process for thread 4  49 64 15
populate thread process for thread 5  65 80 15
populate thread process for thread 6  81 100 19
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5
start process thread 6


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This 

In [9]:
full_model_data = ModelDataLoader('003')

In [10]:
full_model_data.df.count()

index                                 8991
track_id                              8991
dataset                               8991
audio_path                            8991
label                                 8991
fma_genre_top                         8991
fma_genres                            8991
fma_genres_all                        8991
file_available                        8991
sampling_rate                         8991
features                              8991
spectral_centroids_mean               8991
spectral_centroids_delta_mean         8991
spectral_centroids_accelerate_mean    8991
spectral_bandwidth_mean               8991
spectral_rolloff_mean                 8991
zero_crossing_rate_mean               8991
rms_mean                              8991
chroma_stft_mean                      8991
mfccs_mean                            8991
onset                                 8991
tempo                                 8991
contrast                              8991
tonnetz    

In [7]:
full_model_data.df.head()

,index,track_id,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,file_available,sampling_rate,...,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean,onset,tempo,contrast,tonnetz,mfccs_min,mfccs_max
0,0,000002,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.098364,0.141225,0.424169,-1.742610,1.403928,166.708664,21.300203,0.022087,-505.499176,196.868652
1,2,000005,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.059334,0.145243,0.475249,1.318397,1.384270,99.384018,21.096722,0.005648,-528.116272,209.443054
2,3,000010,fma,/project_data_source/free_music_archive/fma_sm...,pop,Pop,[10],[10],1.0,44100,...,0.080989,0.186656,0.430406,2.244156,1.123551,112.347145,20.852177,-0.033316,-537.197205,229.155151
3,15,000140,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.027256,0.068459,0.450882,-3.624865,1.135973,107.666016,21.727451,0.012266,-592.895813,199.950806
4,16,000141,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.034120,0.100852,0.341356,-4.471582,0.881413,120.185318,23.654802,-0.003315,-536.546936,222.208649


## Extraction while saving audio and features on small sample
in this run, we saved the output of librosa load in the parquet as well.  This takes a lot of memory so only did a small example.   This is also a demo of using AudioFeatureExtractor directly for a single extraction on a subset of the data.  

In [4]:
#instantiate data 

gtza_test_data = df_filtered[df_filtered.dataset == 'gtzan']

In [5]:
#instantiate extractor
#commented in alternate return classe in get_audio_data method to include full audio
gtza_extract = AudioFeatureExtractor(gtza_test_data.head(5))

In [6]:
#run extraction
#print statement uncommented 
gtza_extract.add_audio_data_to_df()

Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00000.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00001.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00002.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00003.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00004.wav
Run Librosa Load
extracting_features
putting features to their own columns


In [7]:
#save the results
gtza_extract.save_results() 

In [10]:
#Load the model data for this version
model_data = ModelDataLoader('000')

In [11]:
#Note full librosa_load output array was also saved in this parquet
model_data.df

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,librosa_load,sampling_rate,features,spectral_centroids_mean,spectral_centroids_delta_mean,spectral_centroids_accelerate_mean,spectral_bandwidth_mean,spectral_rolloff_mean,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean
track_id,,,,,,,,,,,,,,,,,,
blues.00000.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0073242188, 0.016601562, 0.0076293945, -0.0...",22050,"[1784.1227, -0.49052292, -0.061881434, 2002.41...",1784.122681,-0.490523,-0.061881,2002.412354,3805.723145,0.083045,0.130184,0.350129,3.121230
blues.00001.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0034179688, 0.0043029785, 0.001373291, 0.00...",22050,"[1530.2617, 0.6459638, -0.0855879, 2038.9875, ...",1530.261719,0.645964,-0.085588,2038.987549,3550.713623,0.056040,0.095908,0.340849,-0.007777
blues.00002.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.019012451, 0.047698975, 0.029418945, -0.018...",22050,"[1552.8325, 0.3400274, 0.025039379, 1747.754, ...",1552.832520,0.340027,0.025039,1747.754028,3042.410156,0.076291,0.175473,0.363538,3.364070
blues.00003.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.013000488, -0.03060913, -0.036071777, -0.0...",22050,"[1070.1534, 0.25125474, 0.07936938, 1596.4226,...",1070.153442,0.251255,0.079369,1596.422607,2184.879150,0.033309,0.141040,0.404854,0.866870
blues.00004.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.0063171387, -0.009277344, -0.008331299, -0...",22050,"[1835.1285, -0.17207043, -0.24526799, 1748.410...",1835.128540,-0.172070,-0.245268,1748.410767,3579.957520,0.101461,0.091501,0.308526,-9.653034
